### Introduction

In this notebook, you'll learn how to:
- Chat with your second brain
- Use LangChain's `ObsidianLoader`
- Use LangChain's `VectorstoreIndexCreator`


#### Installing packages
Before we start, you need to have a few Python libraries installed. You can install these libraries by running the following command in your terminal:
```bash
pip install openai langchain python-dotenv chromadb tiktoken
```

### Loading API Keys

In [1]:
import os
from dotenv import load_dotenv

# load_dotenv()

# Get the absolute path of the current script
script_dir = os.path.abspath(os.getcwd())

# Get the absolute path of the parent directory
parent_dir = os.path.join(script_dir, os.pardir)

dotenv_path = os.path.join(parent_dir, '.env')
# Load the .env file from the parent directory
load_dotenv(dotenv_path)

True

### Loading Notes from Obsidian

Just give it a path

In [2]:
from langchain.document_loaders import ObsidianLoader

loader = ObsidianLoader('/home/kris/Documents/SmartNotes/SecondBrain/Reference Notes/')
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
# docs[:5]

In [4]:
for doc in docs[:5]:
    print(doc.metadata["source"])

📚 Show Your Work.md
📚 Storyworthy.md
📚 The Subtle Art of Not Giving a F.ck.md
📚 On Writing Well.md
📚 Atomic Habits.md


### Create Vector database

Creating the vector database in Chroma.

In [5]:
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator(
    vectorstore_kwargs={"persist_directory": "./chroma/obsidian"}
    ).from_documents(docs)

### Querying My Second Brain

What happens here?
1. Our query gets converted into vector embeddings
2. We search through the vector database to find the most similar documents (that potentially include the answer to our query)
3. We send our query together with the relevant documents to our Large Language Model.
4. LLM send the response - our `answer`

In [6]:
query = "What are 4 laws of behavior change?"
# expecting Atomic Habits
index.query_with_sources(query)

{'question': 'What are 4 laws of behavior change?',
 'answer': ' The four laws of behavior change are: 1. Make it Obvious, 2. Make it Attractive, 3. Make it Easy, and 4. Make it Satisfying.\n',
 'sources': '📚 Atomic Habits.md'}

#### Helpful Function for showing the results

In [7]:
def display_answer(answer):
    print("Sources: ", answer["sources"], "\n\n", answer["answer"])

In [8]:
query = "Give me 5 key takeaways from $100M Offers"
display_answer(index.query_with_sources(query))

Sources:  📚 $100M Offers.md 

  The key takeaways from $100M Offers are:
1. Sell your products based on VALUE not PRICE.
2. Focus on the value you're providing.
3. Create the "Category of One."
4. Sell in a vacuum.
5. Three levers on Success: Market > Offer > Copywriting.



Halucinations?

In [9]:
query = "What are 10 brain foods?"
display_answer(index.query_with_sources(query))

Sources:  https://www.healthline.com/nutrition/10-brain-foods#section2
https://www.webmd.com/diet/features/eat-smart-healthier-brain#1 

  Brain foods include foods high in omega-3 fatty acids, foods high in antioxidants, foods high in B vitamins, foods high in vitamin E, foods high in vitamin C, dark chocolate, blueberries, turmeric, green tea, and nuts.



Just wrong...

In [10]:
query = "What are 10 brain foods from Limitless?"
display_answer(index.query_with_sources(query))

Sources:  📚 Limitless.md, 📚 Building a Second Brain.md, 📚 Dopamine Nation.md, 📚 Keep Sharp.md 

  10 brain foods from Limitless are: exercise, nutrition, sleep, new learning, social interactions, interconectedness, neuroplasticity, FOMO, self-expression, and chasing what excites you.



In [11]:
query = "What are the pillars of a healthy brain?"
# Expecting Keep Sharp
display_answer(index.query_with_sources(query))

Sources:  📚 Keep Sharp.md 

  The five pillars of a healthy brain are exercise, nutrition, sleep, new learning, and social interactions.



In [12]:
query = "How to create wealth?"
display_answer(index.query_with_sources(query))

Sources:  📚 The Almanack Of Naval Ravikant.md 

  According to Naval Ravikant, creating wealth involves finding and building specific knowledge, building or buying equity in a business, and finding a position of leverage.



Simple, "I don't know."

In [13]:
query = "What does Naval Ravikant say about raising kids?"
display_answer(index.query_with_sources(query))

Sources:  📚 The Boron Letters.md, 📚 The Genius In All Of Us.md, 📚 Dopamine Nation.md 

  Naval Ravikant does not mention raising kids.



In [14]:
query = "What is hedonic adaptation?"
display_answer(index.query_with_sources(query))

Sources:  📚 The Subtle Art of Not Giving a F.ck.md, 📚 The Gap And The Gain.md, 📚 Dopamine Nation.md 

  Hedonic adaptation is a phenomenon in which you achieve your goal only to feel unfulfilled and wanting more, and is caused by repeated exposure to the same or similar pleasure stimulus. It is also known as tolerance, and can lead to a dopamine deficit state.



In [15]:
query = "Why most people aren't successful?"
display_answer(index.query_with_sources(query))

Sources:  📚 The Long Game.md, 📚 The Boron Letters.md, 📚 The 10X Rule.md 

  People are often unsuccessful because they are afraid to set "unrealistic" goals, they fear failure, they are met with negativity from friends and family, and they depend on others instead of themselves.

